In [ ]:
%pip install -r requirements.txt

## 1. Model Architecture

The DocRes model is based on the Restormer architecture, which is a Transformer-based model designed for high-resolution image restoration. It uses a hierarchical structure with the following components:


### Key Components:
- **Transformer Blocks**: Uses self-attention mechanisms with efficient memory usage
- **U-Net-like Structure**: Contains encoder and decoder with skip connections
- **Patch Embedding**: Processes the input image as overlapping patches
- **Multi-head Attention**: With adjustable number of heads at different levels
- **Feed-Forward Networks**: Using gated depth-wise convolutions


### Parameters:
- Input: 6-channel image (3 RGB channels + 3 auxiliary channels for processing)
- Output: 3-channel enhanced image
- Base dimension: 24 channels
- Transformer blocks: [1,1,2,2] for different encoder/decoder levels
- Attention heads: [1,2,2,4] for different levels
- Refinement blocks: 2
- FFN expansion factor: 1.5


## 2. Data Preprocessing



The DocRes model processes document images through several preprocessing steps:

### Training Data Preparation:
1. **Document Images**: Input and ground truth pairs
2. **Image Size**: Resized to 256×256 patches for training
3. **Normalization**: Pixel values normalized to [0,1]
4. **Data Augmentation**: 
   - Random cropping
   - Random rotation and scaling
   - Color and contrast adjustments
   - Simulated document degradations (shadows, bleed-through)

### For Appearance Enhancement:
1. Images are processed through a specialized pipeline that enhances document appearance
2. The model is trained with 6 input channels:
   - RGB image (3 channels)
   - Pre-processed enhancement guidance (3 channels)

## 3. Training Process

### Setup:
1. **Environment**: PyTorch with CUDA support
2. **Optimizer**: AdamW with learning rate of 2e-4 and weight decay of 5e-4
3. **Loss Function**: L1 loss between prediction and ground truth
4. **Scheduler**: CosineAnnealing learning rate scheduler
5. **Training Duration**: 300,000 iterations

### Memory Optimizations:
1. **Gradient Checkpointing**: Reduces memory usage by recomputing intermediate activations
2. **Mixed Precision Training**: Uses AMP (Automatic Mixed Precision)
3. **Memory Allocation Config**: `PYTORCH_CUDA_ALLOC_CONF='max_split_size_mb:32,expandable_segments:True'`
4. **Gradient Clipping**: `max_norm=1.0`
5. **Efficient Attention**: Using PyTorch's memory-efficient attention


to check the logs and the val inference command `tensordoard --logdir log`

In [ ]:
!python train.py --experiment_name appearance_enhancement --logdir logs --im_size 256 --batch_size 4 --l_rate 2e-4 --total_iter 65000 --tboard True


iter [10/65000] -- loss: 0.3185 -- lr: 0.000200 -- time remaining: 31 H : 6 Min
iter [20/65000] -- loss: 0.1686 -- lr: 0.000200 -- time remaining: 31 H : 12 Min
iter [30/65000] -- loss: 0.1193 -- lr: 0.000200 -- time remaining: 100 H : 54 Min
iter [40/65000] -- loss: 0.1569 -- lr: 0.000200 -- time remaining: 85 H : 55 Min
iter [50/65000] -- loss: 0.1393 -- lr: 0.000200 -- time remaining: 27 H : 31 Min
iter [60/65000] -- loss: 0.0696 -- lr: 0.000200 -- time remaining: 28 H : 6 Min
iter [70/65000] -- loss: 0.0844 -- lr: 0.000200 -- time remaining: 132 H : 20 Min
iter [80/65000] -- loss: 0.1690 -- lr: 0.000200 -- time remaining: 108 H : 52 Min
iter [90/65000] -- loss: 0.2716 -- lr: 0.000200 -- time remaining: 81 H : 16 Min
iter [100/65000] -- loss: 0.1157 -- lr: 0.000200 -- time remaining: 33 H : 22 Min
iter [110/65000] -- loss: 0.1199 -- lr: 0.000200 -- time remaining: 33 H : 16 Min
iter [120/65000] -- loss: 0.1785 -- lr: 0.000200 -- time remaining: 96 H : 29 Min
iter [130/65000] -- los

In [16]:
!python inference.py --input MannualDataset/DocAligner_results/2019_mobile_i6_foff_sC_PR_028_in.jpg --output result.jpg --checkpoint logs/appearance_enhancement/checkpoint_65000.pkl --patch_size 256 --overlap 32 

Using device: cuda
/media/usamah/84a2f44c-6dd1-4271-ad93-7d9facc4eec4/DocRes_to_tflite_on_enhancement/inference.py:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoi

## 4. Converting to TensorFlow

The PyTorch model is converted to TensorFlow in three steps:
### Step 1: PyTorch to ONNX
```bash
python doc_tf_modules/convert/convert_to_tensorflow.py --checkpoint [CHECKPOINT_PATH] --onnx_path docres_model.onnx --tf_dir tensorflow_model --input_size 256
```

This process:
1. Loads the PyTorch model and weights
2. Creates a dummy input tensor of shape [1, 6, 256, 256]
3. Exports to ONNX format with dynamic batch dimension
4. Tests the ONNX model for correctness

### Step 2: ONNX to TensorFlow
The same script continues to:
1. Load the ONNX model
2. Convert to TensorFlow SavedModel format
3. Save to specified directory
4. Test the TensorFlow model for correctness
5. Create a TensorFlow inference script

In [4]:
!python doc_tf_modules/convert/convert_to_tensorflow.py --checkpoint logs/appearance_enhancement/checkpoint_65000.pkl --onnx_path doc_tf_modules/saved_models/docres_model.onnx --tf_dir doc_tf_modules/saved_models/tensorflow_model

2025-04-09 01:21:57.133541: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 01:21:57.162683: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-09 01:21:57.162755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-09 01:21:57.163510: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 01:21:57.167891: I tensorflow/core/platform/cpu_feature_guar

In [15]:
!python doc_tf_modules/inference/tf_inference.py --model_dir doc_tf_modules/saved_models/tensorflow_model --input MannualDataset/DocAligner_results/2019_mobile_i6_foff_sC_PR_028_in.jpg --output result_tf.jpg --patch_size 320 --overlap 64

2025-04-09 01:58:01.516290: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 01:58:01.544335: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-09 01:58:01.544368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-09 01:58:01.545220: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 01:58:01.549840: I tensorflow/core/platform/cpu_feature_guar

## 5. Converting to TFLite

### Standard TFLite Conversion:
```bash
python doc_tf_modules/convert/convert_to_tflite.py --saved_model_dir doc_tf_modules/saved_models/tensorflow_model --tflite_model_path doc_tf_modules/saved_models/tflite_models_saved/docres_model.tflite
```

This process:
1. Loads the TensorFlow SavedModel
2. Configures the TFLite converter with:
   - TF Select operations support
   - Default optimizations
   - Experimental features for better performance
3. Converts and saves the TFLite model
4. Creates an inference script for the TFLite model

In [5]:
!python doc_tf_modules/convert/convert_to_tflite.py --saved_model_dir doc_tf_modules/saved_models/tensorflow_model --tflite_model_path doc_tf_modules/saved_models/tflite_models_saved/docres_model.tflite

2025-04-09 01:24:35.143596: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 01:24:35.171370: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-09 01:24:35.171403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-09 01:24:35.172169: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 01:24:35.176882: I tensorflow/core/platform/cpu_feature_guar

In [9]:
!python doc_tf_modules/inference/tflite_inference.py --input MannualDataset/DocAligner_results/2019_mobile_i6_foff_sC_PR_028_in.jpg --output result_tflite.jpg --model doc_tf_modules/saved_models/tflite_models_saved/docres_model.tflite --patch_size 256 --overlap 64

2025-04-09 01:32:49.493595: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 01:32:49.564807: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-09 01:32:49.564866: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-09 01:32:49.566742: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 01:32:49.578071: I tensorflow/core/platform/cpu_feature_guar

### Float16 Quantization:
```bash
python doc_tf_modules/convert/convert_to_tflite_float16.py --saved_model_dir doc_tf_modules/saved_models/tensorflow_model --tflite_model_path doc_tf_modules/saved_models/tflite_models_saved/docres_model_float16.tflite
```

Additional configurations for float16:
1. Sets `target_spec.supported_types = [tf.float16]`
2. Keeps input/output as float32 for better compatibility
3. Achieves approximately 46% size reduction (8.50MB → 4.59MB)

In [6]:
!python doc_tf_modules/convert/convert_to_tflite_float16.py --saved_model_dir doc_tf_modules/saved_models/tensorflow_model --tflite_model_path doc_tf_modules/saved_models/tflite_models_saved/docres_model_float16.tflite

2025-04-09 01:27:20.467810: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 01:27:20.495879: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-09 01:27:20.495914: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-09 01:27:20.496693: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 01:27:20.501394: I tensorflow/core/platform/cpu_feature_guar

In [10]:
!python doc_tf_modules/inference/tflite_inference_float16.py --input MannualDataset/DocAligner_results/2019_mobile_i6_foff_sC_PR_028_in.jpg --output result_tflite_float_16.jpg --model doc_tf_modules/saved_models/tflite_models_saved/docres_model_float16.tflite --patch_size 256 --overlap 64

2025-04-09 01:37:21.016211: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 01:37:21.048245: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-09 01:37:21.048276: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-09 01:37:21.049329: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 01:37:21.054363: I tensorflow/core/platform/cpu_feature_guar

In [12]:
!python doc_tf_modules/analyze_tflite_model.py  doc_tf_modules/saved_models/tflite_models_saved/docres_model.tflite doc_tf_modules/saved_models/tflite_models_saved/docres_model_float16.tflite --compare

2025-04-09 01:46:56.526084: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 01:46:56.551295: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-09 01:46:56.551321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-09 01:46:56.552010: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-09 01:46:56.555986: I tensorflow/core/platform/cpu_feature_guar